#### Run the Vectorize.py file

# Isolated Word Recognition

Use the Train sets to build a average template model and use the dev set as a testing set 

In [1]:
import numpy as np
import pickle

from DTW import DTW_distance,DTW_template_alignment,DTW_align

# import data dictionary
with open('Team7_data.txt','rb') as f:
    Data = pickle.load(f)
    f.close()

print(Data.keys())

dict_keys(['2_dev', '2_train', '3_dev', '3_train', '4_dev', '4_train', '5_dev', '5_train', 'z_dev', 'z_train'])


### Build Average Template Model

These used as samples to measure DTW distance cost against

In [2]:
Avg_template_model = []
for name in ['2_train', '3_train', '4_train', '5_train', 'z_train']:
    dx = Data[name]
    wrd_temps = [dx[i] for i in dx]

    # Choose Master Template
    # t = np.argmin([i.shape[1] for i in wrd_temps])      # best for z and 5
    t = np.argsort([i.shape[1] for i in wrd_temps])[len(wrd_temps)//2]  # best for 4
    mstr_temp = wrd_temps[t]
    avg_temp = DTW_template_alignment(mstr_temp,wrd_temps)
    Avg_template_model.append(avg_temp)
    # Debug
    print(f'Number of Templates in {name}: {len(wrd_temps)}')
    print(f"Index for master template: {t}")
    print(f"Name of master Template: {list(dx.keys())[t]}\n")


print(f"Number of General Templates: {len(Avg_template_model)}")
print("Template Shapes:")
for t in Avg_template_model:
    print(t.shape)

Number of Templates in 2_train: 39
Index for master template: 5
Name of master Template: bi_2.wav

Number of Templates in 3_train: 39
Index for master template: 13
Name of master Template: ea_3.wav

Number of Templates in 4_train: 39
Index for master template: 22
Name of master Template: ig_4.wav

Number of Templates in 5_train: 39
Index for master template: 1
Name of master Template: ag_5.wav

Number of Templates in z_train: 39
Index for master template: 4
Name of master Template: bh_z.wav

Number of General Templates: 5
Template Shapes:
(42, 51)
(42, 54)
(42, 50)
(42, 52)
(42, 79)


In [3]:
def IWR(avg_model,targets,ytrue):
    """

    For each target perform DTW with all templates inside avg_model
    Args:
        avg_model:
                List of general templates of words
        targets:
            a dictionary of targets MFCC vectors
    Return:
        DTW distance scores and the closely related word
    """

    Results = []
    acc = 0
    # t_tags = list(targets.keys())
    s_tags = ['2','3','4','5','z']
    for t_tag in targets:
        cost = []
        for m, temp in zip(s_tags,avg_model):
            # Measure distance cost of target against template
            c = DTW_distance(temp,targets[t_tag])
            cost.append(c)
        # Get the Closest Match
        a = np.argmin(np.array(cost))
        #Update Results:
        Results.append((t_tag,s_tags[a],round(cost[a],3)))
        if s_tags[a]==ytrue:
            acc+=1
    return Results,round(acc/len(Results),3)

In [4]:
targets = Data['2_dev']
R,a = IWR(Avg_template_model,targets,'2')
print(f"Model Accuracy: {a}")
print(f'Results:')
for i in R:
    print(i)

Model Accuracy: 0.167
Results:
('mk_2.wav', '4', 48382694.004)
('mm_2.wav', '4', 24194766.948)
('ms_2.wav', '4', 96384802.112)
('mw_2.wav', 'z', 776106509.513)
('nc_2.wav', 'z', 19346086.467)
('ng_2.wav', '4', 10688565.16)
('nh_2.wav', '2', 25355878.797)
('pe_2.wav', '4', 44389906.964)
('pk_2.wav', '3', 9497080.039)
('pm_2.wav', '4', 20268396.731)
('pp_2.wav', '3', 15984120.524)
('ra_2.wav', '2', 66064533.244)


In [5]:
targets = Data['3_dev']
R,a = IWR(Avg_template_model,targets,'3')
print(f"Model Accuracy: {a}")
print(f'Results:')
for i in R:
    print(i)

Model Accuracy: 0.167
Results:
('mk_3.wav', 'z', 19277896.658)
('mm_3.wav', '3', 6559347.777)
('ms_3.wav', '4', 24981263.798)
('mw_3.wav', 'z', 115532047.692)
('nc_3.wav', 'z', 10521501.327)
('ng_3.wav', 'z', 14107841.248)
('nh_3.wav', '2', 25067483.44)
('pe_3.wav', '2', 30539103.539)
('pk_3.wav', '3', 10991219.817)
('pm_3.wav', '4', 20296586.6)
('pp_3.wav', '4', 12355005.365)
('ra_3.wav', 'z', 10778392.409)


In [6]:
targets = Data['4_dev']
R,a = IWR(Avg_template_model,targets,'4')
print(f"Model Accuracy: {a}")
print(f'Results:')
for i in R:
    print(i)

Model Accuracy: 0.75
Results:
('mk_4.wav', 'z', 220607882.581)
('mm_4.wav', '4', 25815619.519)
('ms_4.wav', '4', 62649251.992)
('mw_4.wav', '4', 52829514.175)
('nc_4.wav', '4', 16883926.997)
('ng_4.wav', '5', 6685985.862)
('nh_4.wav', '4', 31297391.311)
('pe_4.wav', 'z', 22733890.756)
('pk_4.wav', '4', 16006387.842)
('pm_4.wav', '4', 10180847.668)
('pp_4.wav', '4', 12111203.574)
('ra_4.wav', '4', 35095431.977)


In [7]:
targets = Data['5_dev']
R,a = IWR(Avg_template_model,targets,'5')
print(f"Model Accuracy: {a}")
print(f'Results:')
for i in R:
    print(i)

Model Accuracy: 0.083
Results:
('mk_5.wav', 'z', 170585743.482)
('mm_5.wav', '4', 16612379.046)
('ms_5.wav', 'z', 503394462.545)
('mw_5.wav', 'z', 207719851.681)
('nc_5.wav', '4', 12195236.414)
('ng_5.wav', '5', 9482565.617)
('nh_5.wav', '4', 31317882.176)
('pe_5.wav', '4', 4953265.692)
('pk_5.wav', '3', 8851328.981)
('pm_5.wav', '4', 46022881.564)
('pp_5.wav', '4', 14749394.58)
('ra_5.wav', '4', 14961327.127)


In [8]:
targets = Data['z_dev']
R,a = IWR(Avg_template_model,targets,'z')
print(f"Model Accuracy: {a}")
print(f'Results:')
for i in R:
    print(i)

Model Accuracy: 0.583
Results:
('mk_z.wav', 'z', 59560592.883)
('mm_z.wav', '4', 41457712.629)
('ms_z.wav', 'z', 65420906.703)
('mw_z.wav', 'z', 145492456.82)
('nc_z.wav', '2', 54950830.782)
('ng_z.wav', 'z', 11497222.056)
('nh_z.wav', '2', 45376746.856)
('pe_z.wav', 'z', 55201160.731)
('pk_z.wav', 'z', 11682309.655)
('pm_z.wav', 'z', 10125173.827)
('pp_z.wav', '4', 25907022.406)
('ra_z.wav', '4', 17568233.364)
